<a href="https://colab.research.google.com/github/ShubhamShaswat/Toxicity-Analysis/blob/master/kaggle_toxicity_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from google.colab import files

from keras.preprocessing import text,sequence
from tqdm import tqdm
from gensim.models import KeyedVectors

Using TensorFlow backend.


In [0]:

uploaded = files.upload()
!pip install kaggle
#Copy key to the relevant folder
!cp kaggle.json /root/.kaggle/kaggle.json
#Download dataset
!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

Saving kaggle.json to kaggle (1).json
  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 68.9MB/s]
 41% 5.00M/12.1M [00:00<00:00, 44.9MB/s]
100% 12.1M/12.1M [00:00<00:00, 76.3MB/s]
 97% 265M/273M [00:01<00:00, 99.4MB/s]
100% 273M/273M [00:02<00:00, 141MB/s] 


In [0]:

!ls

sample_data


In [0]:
import requests, zipfile, io
r = requests.get('http://nlp.stanford.edu/data/glove.840B.300d.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

KeyboardInterrupt: ignored

In [0]:
#!unzip sample_submission.csv.zip
!unzip train.csv.zip
#!unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
train=pd.read_csv('train.csv')
#df_test=pd.read_csv('test.csv')
#sample=pd.read_csv('sample_submission.csv')


In [0]:
TEXT_COLUMN='comment_text'
TARGET='target'
x_train=train[TEXT_COLUMN]
y_train=train[TARGET]
del train

In [0]:
tokenizer=text.Tokenizer()
tokenizer.fit_on_texts(list(x_train))

In [0]:
x_train=tokenizer.texts_to_sequences(x_train)

In [0]:
x_train=sequence.pad_sequences(x_train,maxlen=250)

In [0]:
miss_val=df_train.isnull().sum(axis=0)/len(df_train)
miss_val_df=miss_val[miss_val>0]*100


In [0]:
identities=list(miss_val_df.keys()[:-1])


In [0]:
train_df=df_train.loc[:,['target']+identities].dropna()


In [0]:
toxic_df = train_df[train_df['target'] >= .5][identities]
non_toxic_df = train_df[train_df['target'] < .5][identities]

In [0]:
toxic_count = toxic_df.where(train_df == 0, other = 1).sum()
non_toxic_count = non_toxic_df.where(train_df == 0, other = 1).sum()

In [0]:
total_toxic_count=len(toxic_df)

In [0]:
comments=[comment.strip().split(' ') for comment in df_train[df_train['target'] >= .5]['comment_text'].dropna()]

In [0]:
words=[]
for c in comments:
  for w in c:
    words.append(w.lower())


In [0]:
words_set=list(set(words))
words_freq = {w:0 for w in words_set}
for w in words:
  words_freq[w]+=1



In [0]:
for w in words_freq.keys():
  words_freq[w] /= len(words)
  words_freq[w]*100
  

In [0]:
new={}
for k,v in words_freq.items():
  if v <= 5.0e:
    new[k]=v

In [0]:
for i in identities:
  try:
    print(i,' : ',words_freq[i.lower()]*1.0e5)
  except KeyError:
    pass

asian  :  2.207635162816631
atheist  :  1.0613630590464573
bisexual  :  0.2547271341711497
black  :  73.85671740217948
buddhist  :  0.42454522361858293
christian  :  14.009992379413237
female  :  8.561662009641422
heterosexual  :  1.500059790118993
hindu  :  0.29718165653300804
jewish  :  5.335118310140191
latino  :  0.8207874323292603
male  :  12.665599171287722
muslim  :  31.40219504032118
transgender  :  3.764300982751435
white  :  142.5764375985741


In [0]:
plt.bar(np.arange(25),toxic_count,0.35)
plt.bar(np.arange(25),non_toxic_count,0.35,bottom=toxic_count)
plt.xlabel('Toxic Vs Non Toxic')
plt.ylabel('Counts')
plt.xticks(np.arange(25),identities)

In [0]:
toxic_vs_non_toxic = pd.concat([toxic_count, non_toxic_count], axis=1)


In [0]:
del train

In [0]:

def get_coeffs(word,*arr):
  
  return word,np.asarray(arr,dtype=np.float32)

def load_embeddings(path):
  embedding_index={}
  with open(path) as f:
    for line in tqdm(f):
      val=line.strip().split(' ')
      embedding_index[val[0]]=np.asarray(val[1:],dtype=np.float32)
  return embedding_index    
  
  
def build_matrix(word_index,path):
  
  embedding_index=load_embeddings(path)
  #embedding_index = KeyedVectors.load(path, mmap='r')

  embedding_matrix=np.zeros((len(word_index)+1,300))
  
  for word,i in tqdm(word_index.items()):
    try:
      embedding_matrix[i]=embedding_index[word]
    except KeyError:
      pass
  del embedding_index  
  return embedding_matrix
      
  

In [0]:
path='glove.840B.300d.txt'



array([ 0.98 , 98.   ,  0.765], dtype=float32)

In [0]:

embedding_matrix=build_matrix(tokenizer.word_index,path)

2196017it [03:25, 10699.64it/s]
100%|██████████| 397708/397708 [00:00<00:00, 440319.88it/s]


In [0]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D

In [0]:
NUM_MODELS = 2
BATCH_SIZE = 512
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
EPOCHS = 4
MAX_LEN=250

def build_model(embedding_matrix,num_aux_targets):
  words=Input(shape=(MAX_LEN,))
  x=Embedding(*embedding_matrix.shape,weights=[embedding_matrix],trainable=False)(words)
  x=SpatialDropout1D(0.3)(x)
  x=Bidirectional(CuDNNLSTM(LSTM_UNITS,return_sequences=True))(x)
  x=Bidirectional(CuDNNLSTM(LSTM_UNITS,return_sequences=True))(x)
  
  hidden = concatenate([
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x),
    ])
  hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
  hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
  result = Dense(1, activation='sigmoid')(hidden)
  #aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)
    
  model = Model(inputs=words, outputs=result)
  model.compile(loss='binary_crossentropy', optimizer='adam')

  return model
  


In [0]:
model=build_model(embedding_matrix,1)
model.fit(x_train,y_train, batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=2,)
  

NameError: ignored